In [6]:
import pandas as pd
import altair as alt

In [7]:
df = pd.read_csv('data/SP500_merged.csv')
df.head()

,Date,Close,Volume,Symbol,GICS Sector,GICS Sub-Industry
0,2022-01-03 00:00:00-05:00,177.740005,1930700,MMM,Industrials,Industrial Conglomerates
1,2022-01-04 00:00:00-05:00,180.229996,2522200,MMM,Industrials,Industrial Conglomerates
2,2022-01-05 00:00:00-05:00,179.490005,2952400,MMM,Industrials,Industrial Conglomerates
3,2022-01-06 00:00:00-05:00,178.000000,2505400,MMM,Industrials,Industrial Conglomerates
4,2022-01-07 00:00:00-05:00,179.949997,2800200,MMM,Industrials,Industrial Conglomerates


In [8]:
# Obtain the sector name as a list, total 11 sectors
sector_name = df['GICS Sector'].unique().tolist()

sector_name

['Industrials',
 'Health Care',
 'Information Technology',
 'Communication Services',
 'Consumer Staples',
 'Consumer Discretionary',
 'Utilities',
 'Financials',
 'Materials',
 'Real Estate',
 'Energy']

In [81]:
# Obtain the symbol of each sector as a list
sector_symbol = []
for i in sector_name:
    sector_symbol.append(df[df['GICS Sector']==i]['Symbol'].unique().tolist())

#sector_symbol

In [28]:
# Define a function to calculate the growth rate of each company (sector_symbol) in each sector (sector_name)
def growth_rate_company(duration):
    company_growth_rate = []

    for i in range(len(sector_name)):
        grow_rate = []

        for n in sector_symbol[i]:
            dff = df[df['GICS Sector'] == sector_name[i]][df['Symbol'] == n]
            grow_rate.append(dff.iloc[len(dff) - 1, 1] / dff.iloc[len(dff) - 1 - duration, 1] - 1)

        company_growth_rate.extend([{'sector': sector_name[i],
                                     'company': n, 'growth_rate': g
                                     } for n,g in zip(sector_symbol[i],grow_rate)])

    company_growth_rate = pd.DataFrame(company_growth_rate)

    return company_growth_rate


In [29]:
growth_rate_company(90)

/var/folders/7s/086nw8893dq343kp9p40cjdw0000gn/T/ipykernel_10419/1781933829.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  dff = df[df['GICS Sector'] == sector_name[i]][df['Symbol'] == n]


,sector,company,growth_rate
0,Industrials,MMM,-0.061406
1,Industrials,AOS,0.272798
2,Industrials,ALK,0.135569
3,Industrials,ALLE,0.188485
4,Industrials,AAL,0.163504
...,...,...,...
496,Energy,PXD,-0.177313
497,Energy,SLB,0.247002
498,Energy,TRGP,0.112601
499,Energy,VLO,0.122260


In [58]:
company_growth_rate = growth_rate_company(90)

# Filter the top 5 companies in each sector
top_5 = company_growth_rate.groupby('sector').apply(lambda x: x.nlargest(5, 'growth_rate'))
top_5 = top_5.reset_index(drop=True)
top_5

/var/folders/7s/086nw8893dq343kp9p40cjdw0000gn/T/ipykernel_10419/1781933829.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  dff = df[df['GICS Sector'] == sector_name[i]][df['Symbol'] == n]


,sector,company,growth_rate
0,Communication Services,NFLX,0.347048
1,Communication Services,META,0.327635
2,Communication Services,OMC,0.312861
3,Communication Services,IPG,0.215529
4,Communication Services,CMCSA,0.211545
5,Consumer Discretionary,WYNN,0.909260
6,Consumer Discretionary,LVS,0.596234
7,Consumer Discretionary,BWA,0.476107
8,Consumer Discretionary,RCL,0.464551
9,Consumer Discretionary,BKNG,0.431258


In [52]:
 # Obtain the symbol of top 5 companies in each sector

top_5_symbol = []
for i in sector_name:
    top_5_symbol.append(top_5[top_5['sector']==i]['company'].unique().tolist())

top_5_symbol

[['URI', 'GE', 'BA', 'TDG', 'UAL'],
 ['ALGN', 'IDXX', 'UHS', 'GEHC', 'XRAY'],
 ['NVDA', 'AMAT', 'MPWR', 'SEDG', 'LRCX'],
 ['NFLX', 'META', 'OMC', 'IPG', 'CMCSA'],
 ['LW', 'CLX', 'CHD', 'EL', 'PM'],
 ['WYNN', 'LVS', 'BWA', 'RCL', 'BKNG'],
 ['EIX', 'PNW', 'ATO', 'PCG', 'PEG'],
 ['ACGL', 'MKTX', 'STT', 'RE', 'BEN'],
 ['STLD', 'FCX', 'NUE', 'DD', 'CE'],
 ['VTR', 'EQIX', 'SPG', 'WELL', 'CBRE'],
 ['BKR', 'SLB', 'HAL', 'OKE', 'MPC']]

In [53]:
# Subset the df to only include the top 5 companies in each sector
df_top_5 = df[df['Symbol'].isin(top_5_symbol[0]+top_5_symbol[1]+top_5_symbol[2]+top_5_symbol[3]+top_5_symbol[4]+top_5_symbol[5]+top_5_symbol[6]+top_5_symbol[7]+top_5_symbol[8]+top_5_symbol[9]+top_5_symbol[10])]

df_top_5

,Date,Close,Volume,Symbol,GICS Sector,GICS Sub-Industry
5220,2022-01-03 00:00:00-05:00,648.049988,428200,ALGN,Health Care,Health Care Supplies
5221,2022-01-04 00:00:00-05:00,623.500000,552000,ALGN,Health Care,Health Care Supplies
5222,2022-01-05 00:00:00-05:00,578.840027,980000,ALGN,Health Care,Health Care Supplies
5223,2022-01-06 00:00:00-05:00,567.619995,1149400,ALGN,Health Care,Health Care Supplies
5224,2022-01-07 00:00:00-05:00,546.989990,1190500,ALGN,Health Care,Health Care Supplies
...,...,...,...,...,...,...
143002,2023-02-22 00:00:00-05:00,107.669998,3375500,WYNN,Consumer Discretionary,Casinos & Gaming
143003,2023-02-23 00:00:00-05:00,105.839996,1820700,WYNN,Consumer Discretionary,Casinos & Gaming
143004,2023-02-24 00:00:00-05:00,105.529999,1942900,WYNN,Consumer Discretionary,Casinos & Gaming
143005,2023-02-27 00:00:00-05:00,105.139999,2259000,WYNN,Consumer Discretionary,Casinos & Gaming


In [61]:
# Subset the df_top_5 based on the GICS Sector

# for example, if we want to see the top 5 companies in Communication Services sector
df_top_5_communication = df_top_5[df_top_5['GICS Sector']=='Communication Services']
df_top_5_communication

,Date,Close,Volume,Symbol,GICS Sector,GICS Sub-Industry
34220,2022-01-03 00:00:00-05:00,50.740002,20706000,CMCSA,Communication Services,Cable & Satellite
34221,2022-01-04 00:00:00-05:00,50.189999,24592800,CMCSA,Communication Services,Cable & Satellite
34222,2022-01-05 00:00:00-05:00,50.220001,20727900,CMCSA,Communication Services,Cable & Satellite
34223,2022-01-06 00:00:00-05:00,50.509998,18288200,CMCSA,Communication Services,Cable & Satellite
34224,2022-01-07 00:00:00-05:00,50.049999,17894500,CMCSA,Communication Services,Cable & Satellite
...,...,...,...,...,...,...
102112,2023-02-22 00:00:00-05:00,91.849998,1954000,OMC,Communication Services,Advertising
102113,2023-02-23 00:00:00-05:00,91.930000,1693800,OMC,Communication Services,Advertising
102114,2023-02-24 00:00:00-05:00,91.599998,1299900,OMC,Communication Services,Advertising
102115,2023-02-27 00:00:00-05:00,91.019997,1829400,OMC,Communication Services,Advertising


In [84]:
# Line chart
line_chart = alt.Chart(df_top_5_communication).mark_line().encode(
    x='Date:T',
    y='Close:Q',
    color='Symbol:N',
    tooltip=['Symbol:N', 'Close:Q']
).properties(
    width=800,
    height=400,
    title='Top 5 Companies by Growth Rate'
).interactive()

line_chart

alt.Chart(...)

In [86]:
# Define a function of line chart with multiple lines to show the trend of top 5 companies in each sector

def line_chart_growth_rate(duration):
    company_growth_rate = growth_rate_company(duration)

    # Filter the top 5 companies in each sector
    top_5 = company_growth_rate.groupby('sector').apply(lambda x: x.nlargest(5, 'growth_rate'))
    top_5 = top_5.reset_index(drop=True)

    # Obtain the symbol of top 5 companies in each sector
    top_5_symbol = []
    for i in sector_name:
        top_5_symbol.append(top_5[top_5['sector']==i]['company'].unique().tolist())

    # Subset the df to only include the top 5 companies in each sector
    df_top_5 = df[df['Symbol'].isin(top_5_symbol[0]+top_5_symbol[1]+top_5_symbol[2]+top_5_symbol[3]+top_5_symbol[4]+top_5_symbol[5]+top_5_symbol[6]+top_5_symbol[7]+top_5_symbol[8]+top_5_symbol[9]+top_5_symbol[10])]

    # Subset the df_top_5 based on the GICS Sector
    # for example, if we want to see the top 5 companies in Communication Services sector
    df_top_5_communication = df_top_5[df_top_5['GICS Sector']=='Communication Services']
    df_top_5_communication

    # Line chart
    line_chart = alt.Chart(df_top_5_communication).mark_line().encode(
        x='Date:T',
        y='Close:Q',
        color='Symbol:N',
        tooltip=['Symbol:N', 'Close:Q']
    ).properties(
        width=800,
        height=400,
        title='Top 5 Companies by Growth Rate'
    ).interactive()

    return line_chart


In [87]:
line_chart_growth_rate(90)

/var/folders/7s/086nw8893dq343kp9p40cjdw0000gn/T/ipykernel_10419/1781933829.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  dff = df[df['GICS Sector'] == sector_name[i]][df['Symbol'] == n]


alt.Chart(...)